<a href="https://colab.research.google.com/github/Gauravrakesh/Multimodal-search-implementation-with-RAG-/blob/main/MultiModal_RAG_using_Vertex_AI_AstraDB(Cassandra)%C2%A0%26%C2%A0Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Vertex AI SDK for Python

In [ ]:
!pip install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install ragstack-ai

In [ ]:
PROJECT_ID = "calm-suprstate-438017-i3"
LOCATION = "us-central1"

In [ ]:
ASTRA_DB_API_ENDPOINT="https://7acbc698-a36e-4707-861e-d12424911fbc-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:XsZIuRgarAaWuPuTBYfcrLXH:1055e205c119979ff1276f15f20124e116e8adc4bd72a41a20d4828079a8df6c"


In [ ]:
import getpass, os, requests

if "GCP_PROJECT_ID" not in os.environ:
  os.environ["GCP_PROJECT_ID"] = getpass.getpass("Provide your GCP Project ID")

if "LOCATION" not in os.environ:
  os.environ["LOCATION"] = getpass.getpass("Provide your GCP Location")

if "ASTRA_DB_ENDPOINT" not in os.environ:
  os.environ["ASTRA_DB_ENDPOINT"] = getpass.getpass("Provide your Astra DB Endpoint")

if "ASTRA_DB_TOKEN" not in os.environ:
  os.environ["ASTRA_DB_TOKEN"] = getpass.getpass("Provide your Astra DB Token")

## Authenticate your notebook environment ( Colab only )

In [ ]:
!gcloud config set project {os.getenv("GCP_PROJECT_ID")}

Updated property [core/project].


In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
!gcloud auth list


No credentialed accounts.

To login, run:
  $ gcloud auth login `ACCOUNT`



## Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
# Define project information
PROJECT_ID=os.getenv("GCP_PROJECT_ID")
LOCATION=os.getenv("LOCATION")

In [ ]:
# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Import libraries

In [ ]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part
)

## Use the Gemini 1.0 Pro model

The Gemini 1.0 Pro (`gemini-1.0-pro`) model is designed to handle natural language tasks, multi-turn text and code chat, and code generation.


## Load the Gemini 1.0 Pro model

In [ ]:
model = GenerativeModel("gemini-1.0-pro")

## Generate text from text prompts

In [ ]:
responses = model.generate_content("Why is the sky blue?", stream=True)

In [ ]:
for response in responses:
    print(response.text, end="")

The sky appears blue due to a phenomenon called Rayleigh scattering. Sunlight, composed of all colors of the rainbow, enters Earth's atmosphere. The atmosphere contains microscopic particles like nitrogen and oxygen molecules that are much smaller than the wavelengths of visible light. 

These particles scatter light in all directions, but they scatter shorter wavelengths (like blue) more effectively than longer wavelengths (like red). As a result, the blue light gets scattered across the sky, reaching our eyes from all directions and making the sky appear blue.

This scattering effect is also why the sun itself appears more yellow or red when it's low in the sky, especially at sunrise or sunset. The sunlight has to travel through more air near the horizon, where it encounters more scattering, which removes even more blue and leaves the longer wavelengths like red and yellow more dominant.


In [ ]:
prompt = """Create a numbered list of 10 items. Each item in the list should be a trend in the tech industry.

Each trend should be less than 5 words."""  # try your own prompt


In [ ]:
responses = model.generate_content(prompt, stream=True)

In [ ]:
for response in responses:
    print(response.text, end="")

## Top 10 Tech Industry Trends:

1. **AI Everywhere:** Continued advancement and integration of artificial intelligence in all aspects of life. 
2. **Quantum Computing:** Leaps in development of powerful quantum computers.
3. **Metaverse Expansion:** Growth and evolution of immersive virtual worlds and digital experiences.
4. **Cybersecurity Focus:** Increased emphasis on protecting data and systems against cyberattacks. 
5. **Personalized Healthcare:** Advancements in personalized medicine and health monitoring through technology.
6. **Sustainable Solutions:** Tech innovations focused on environmental sustainability and climate change. 
7. **Robotic Automation:** Increased adoption of robots for tasks across industries. 
8. **Biotech Breakthroughs:** Advancements in biotechnology, including gene editing and personalized therapies.
9. **Space Exploration:** Increased private and public investment in space exploration and technology.
10. **Decentralized Finance:** Growth of blockchain-b

## Model parameters

Every prompt you send to the model includes parameter values that control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

In [ ]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)


In [ ]:
responses = model.generate_content(
    "Why is the sky blue?",
    generation_config=generation_config,
    stream=True,
)


In [ ]:
for response in responses:
    print(response.text, end="")

The sky is blue because of a phenomenon called **Rayleigh scattering**. 

When sunlight enters the Earth's atmosphere, it interacts with the gas molecules present there. These molecules scatter the sunlight in all directions. However, **blue light is scattered more strongly** than other colors because it has a shorter wavelength. This scattered blue light reaches our eyes from all directions in the sky, making it appear blue.

Here's a breakdown of the process:

1. **Sunlight enters the atmosphere:** Sunlight, which is a mixture of all colors, enters the Earth's atmosphere.
2. **Interaction with gas molecules:** The sunlight interacts with the gas molecules (mainly nitrogen and oxygen) in the atmosphere.
3. **Scattering of light:** The gas molecules scatter the sunlight in all directions.
4. **Blue light scatters more:** Blue light, with its shorter wavelength, is scattered more strongly than other colors.
5. **Blue light reaches our eyes:** The scattered blue light reaches our eyes fr

In [ ]:
source_img_data = requests.get('https://drive.google.com/uc?export=view&id=15ddcn-AIxpvRdWcFGvIr77XLWdo4Maof').content

In [ ]:
with open('coffee_maker_part.png', 'wb') as handler:
  handler.write(source_img_data)

In [ ]:
from langchain_google_vertexai import ChatVertexAI

In [ ]:
from langchain.schema.messages import HumanMessage
from PIL import Image, ImageFile
import os, sys


In [ ]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision",safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },)

In [ ]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "coffee_maker_part.png"},
}
text_message = {
    "type": "text",
    "text": "What is this image? Share a link to purchase a replacement",
}

In [ ]:
message = HumanMessage(content=[text_message, image_message])

In [ ]:
output = chat([message])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
print(output.content)

 This is a basket filter for a Breville coffee maker. You can purchase a replacement here:

https://www.amazon.com/Breville-Replacement-Basket-Filter-800ESXL/dp/B014388O44/ref=sr_1_1?crid=3N980259E167E&keywords=breville+basket+filter&qid=1644981164&sprefix=breville+basket+filter%2Caps%2C146&sr=8-1


In [ ]:
import pandas as pd

d = {'name': ["Saucer", "Saucer Ceramic", "Milk Jug Assembly", "Handle Steam Wand Kit (New Version From 0735 PDC)", "Spout Juice Small (From 0637 to 1041 PDC)", "Cleaning Steam Wand", "Jug Frothing", "Spoon Tamping 50mm", "Collar Grouphead 50mm", "Filter 2 Cup Dual Wall 50mm", "Filter 1 Cup 50mm", "Water Tank Assembly", "Portafilter Assembly 50mm", "Milk Jug Assembly", "Filter 2 Cup 50mm" ],
     'url': ["https://www.breville.com/us/en/parts-accessories/parts/sp0014946.html?sku=SP0014946", "https://www.breville.com/us/en/parts-accessories/parts/sp0014914.html?sku=SP0014914", "https://www.breville.com/us/en/parts-accessories/parts/sp0011391.html?sku=SP0011391", "https://www.breville.com/us/en/parts-accessories/parts/sp0010719.html?sku=SP0010719", "https://www.breville.com/us/en/parts-accessories/parts/sp0010718.html?sku=SP0010718", "https://www.breville.com/us/en/parts-accessories/parts/sp0003247.html?sku=SP0003247", "https://www.breville.com/us/en/parts-accessories/parts/sp0003246.html?sku=SP0003246", "https://www.breville.com/us/en/parts-accessories/parts/sp0003243.html?sku=SP0003243", "https://www.breville.com/us/en/parts-accessories/parts/sp0003232.html?sku=SP0003232", "https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231", "https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230", "https://www.breville.com/us/en/parts-accessories/parts/sp0003225.html?sku=SP0003225", "https://www.breville.com/us/en/parts-accessories/parts/sp0003216.html?sku=SP0003216", "https://www.breville.com/us/en/parts-accessories/parts/sp0001875.html?sku=SP0001875", "https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166"],
     'price': ["10.95", "4.99", "14.95", "8.95", "10.95", "6.95", "24.95", "8.95", "6.95", "12.95", "12.95", "14.95", "10.95", "16.95", "11.95"],
     'image': ["https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014946/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014914/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0011391/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010719/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010718/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003247/tile.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/BES250/SP0003246/SP0003246_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003243/SP0003243_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003232/SP0003232_IMAGE1_400x400.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003225/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003216/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0001875/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg"]}
df = pd.DataFrame(data=d)
df

,name,url,price,image
0,Saucer,https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
1,Saucer Ceramic,https://www.breville.com/us/en/parts-accessori...,4.99,https://www.breville.com/content/dam/breville/...
2,Milk Jug Assembly,https://www.breville.com/us/en/parts-accessori...,14.95,https://www.breville.com/content/dam/breville/...
3,Handle Steam Wand Kit (New Version From 0735 PDC),https://www.breville.com/us/en/parts-accessori...,8.95,https://www.breville.com/content/dam/breville/...
4,Spout Juice Small (From 0637 to 1041 PDC),https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
5,Cleaning Steam Wand,https://www.breville.com/us/en/parts-accessori...,6.95,https://www.breville.com/content/dam/breville/...
6,Jug Frothing,https://www.breville.com/us/en/parts-accessori...,24.95,https://assets.breville.com/cdn-cgi/image/widt...
7,Spoon Tamping 50mm,https://www.breville.com/us/en/parts-accessori...,8.95,https://assets.breville.com/cdn-cgi/image/widt...
8,Collar Grouphead 50mm,https://www.breville.com/us/en/parts-accessori...,6.95,https://assets.breville.com/cdn-cgi/image/widt...
9,Filter 2 Cup Dual Wall 50mm,https://www.breville.com/us/en/parts-accessori...,12.95,https://www.breville.com/content/dam/breville/...


In [ ]:
import vertexai, json, requests
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
from astrapy.db import AstraDB, AstraDBCollection
from google.colab import files

<ipython-input-31-f146adbe4615>:3: DeprecatedWarning: All of 'astrapy.core.*', 'astrapy.api.*', 'astrapy.db.*' and 'astrapy.ops.*' is deprecated as of 1.5.0 and will be removed in 2.0.0. Please refer to https://docs.datastax.com/en/astra-db-serverless/api-reference/dataapiclient.html to start the (recommended) path forward to the current API, and to https://github.com/datastax/astrapy?tab=readme-ov-file#appendix-b-compatibility-with-pre-100-library for details on the deprecated modules.
  from astrapy.db import AstraDB, AstraDBCollection


In [ ]:
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [ ]:
# Initialize our vector db
astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"), api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))

In [ ]:
collection = astra_db.create_collection(collection_name="coffee_shop_ecommerce", dimension=1408)

In [ ]:
for i in range(len(df)):
  name = df.loc[i, "name"]
  image = df.loc[i, "image"]
  price = df.loc[i, "price"]
  url = df.loc[i, "url"]

  # Download this product's image and save it to the Colab filesystem.
  # In a production system this binary data would be stored in Google Cloud Storage
  img_data = requests.get(image).content
  with open(f'{name}.png', 'wb') as handler:
    handler.write(img_data)

  # load the image from filesystem and compute the embedding value
  img = Image.load_from_file(f'{name}.png')
  embeddings = model.get_embeddings(image=img, contextual_text=name)

  try:
    # add to the AstraDB Vector Database
    collection.insert_one({
        "_id": i,
        "name": name,
        "image": image,
        "url": url,
        "price": price,
        "$vector": embeddings.image_embedding,
      })
  except Exception as error:
    # if you've already added this record, skip the error message
    error_info = json.loads(str(error))
    if error_info[0]['errorCode'] == "DOCUMENT_ALREADY_EXISTS":
      print("Document already exists in the database.  Skipping.")

KeyError: 0

In [ ]:
for i in range(len(df)):
  name = df.loc[i, "name"]
  image = df.loc[i, "image"]
  price = df.loc[i, "price"]
  url = df.loc[i, "url"]

  # Download this product's image and save it to the Colab filesystem.
  # In a production system this binary data would be stored in Google Cloud Storage
  img_data = requests.get(image).content
  with open(f'{name}.png', 'wb') as handler:
    handler.write(img_data)

  # load the image from filesystem and compute the embedding value
  img = Image.load_from_file(f'{name}.png')
  embeddings = model.get_embeddings(image=img, contextual_text=name)

  try:
    # add to the AstraDB Vector Database
    collection.insert_one({
        "_id": i,
        "name": name,
        "image": image,
        "url": url,
        "price": price,
        "$vector": embeddings.image_embedding,
      })
  except Exception as error:
    # Check if the error message contains "DOCUMENT_ALREADY_EXISTS"
    if "DOCUMENT_ALREADY_EXISTS" in str(error):
      print("Document already exists in the database.  Skipping.")
    else:
      # If it's a different error, raise it
      raise error

Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.
Document already exists in the database.  Skipping.


In [ ]:
import json

# Embed the similar item
img = Image.load_from_file('coffee_maker_part.png')

In [ ]:
embeddings = model.get_embeddings(image=img, contextual_text="A espresso machine part")

In [ ]:
embeddings.image_embedding

[0.00964636635,
 -0.0426938534,
 0.0018922243,
 0.04409815,
 -0.00553200953,
 -0.0237178989,
 -0.0117072621,
 -0.0151233254,
 -0.0221069548,
 -0.00657869549,
 -0.0138571151,
 -0.0380623601,
 0.0182006303,
 0.159336537,
 -0.0228622984,
 0.00567905186,
 0.0563613214,
 -0.00454364205,
 -0.0366299115,
 0.0169717651,
 -0.0133396368,
 -0.0105286101,
 -0.00489088427,
 -0.0130224153,
 0.000800026814,
 0.00289593497,
 0.00581795163,
 -0.017347116,
 0.0503665023,
 -0.0402288884,
 0.0254745241,
 -0.00334993843,
 -0.00152899127,
 0.0172210094,
 0.0538243204,
 -0.0175644904,
 -0.0202101469,
 0.020622097,
 -0.00948321912,
 -0.00583362859,
 -0.023845192,
 0.0161951371,
 -0.0305653345,
 -0.0224502198,
 0.00977364928,
 -0.0227514263,
 -0.0299170297,
 -0.000240347144,
 0.0094754342,
 -0.00138739438,
 -0.0331848748,
 -0.014900472,
 0.0252219681,
 -0.0311619602,
 0.0339487754,
 0.00024067577,
 -0.0192498788,
 -0.0253038481,
 0.0409211218,
 -0.0407565944,
 0.011477964,
 -0.00957838446,
 0.0339915752,
 -0.0

In [ ]:
# Perform the vector search against AstraDB Vector
documents = collection.vector_find(
    embeddings.image_embedding,
    limit=3,
)

In [ ]:
documents

[{'_id': 14,
  'name': 'Filter 2 Cup 50mm',
  'image': 'https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166',
  'price': '11.95',
  '$similarity': 0.91381836},
 {'_id': 9,
  'name': 'Filter 2 Cup Dual Wall 50mm',
  'image': 'https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231',
  'price': '12.95',
  '$similarity': 0.91381836},
 {'_id': 10,
  'name': 'Filter 1 Cup 50mm',
  'image': 'https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230',
  'price': '12.95',
  '$similarity': 0.9118577}]

In [ ]:
related_products_csv = "name, image, price, url\n"
for doc in documents:
  related_products_csv += f"{doc['name']}, {doc['image']}, {doc['price']}, {doc['url']},\n"

In [ ]:
print(related_products_csv)

name, image, price, url
Filter 2 Cup 50mm, https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg, 11.95, https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166,
Filter 2 Cup Dual Wall 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231,
Filter 1 Cup 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230,



In [ ]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "/content/coffee_maker_part.png"},
}
text_message = {
    "type": "text",
    "text": f"What we have in this image? Share the URL and price to purchase a replacement. Here are related products {related_products_csv}",
}

In [ ]:
message = HumanMessage(content=[text_message, image_message])


In [ ]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision",safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },)

In [ ]:
output = chat([message])

In [ ]:
print(output.content)

 The image shows a Breville 2 Cup Filter. It can be purchased for $11.95 at the following link: https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166

Here are some related products:

Filter 2 Cup Dual Wall 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231
Filter 1 Cup 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230
